## Import Required Packages

In [2]:
import numpy as np
import os
import pandas as pd
import re

from numpy.random import seed
seed(1)

import gensim as gs
import scipy as sc
import nltk
from nltk.tokenize import word_tokenize as wt
from nltk.tokenize import sent_tokenize as st
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# from keras.preprocessing.sequence import pad_sequences
import logging
from collections import Counter

from sklearn.model_selection import train_test_split as tts
import logging
import matplotlib.pyplot as plt
import keras
from keras import backend as k
k.set_learning_phase(1)
from keras import initializers
from keras.optimizers import RMSprop
from keras.models import Model
from keras.layers import Dense,LSTM,Input,Activation,Add,TimeDistributed,Permute,Flatten,RepeatVector,merge,Lambda,Multiply,Reshape
from keras.callbacks import ModelCheckpoint
from keras.models import load_model


C:\Users\Sumi\Anaconda3\envs\NLP\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


## Functions for daily load

In [1]:
def parsetext(dire,category,filename):
    with open("%s\\%s"%(dire+category,filename),'r',encoding="Latin-1") as readin:
#         print("file read successfully")
        text=readin.read()
    return text.lower()


def load_data(dire,category):
    """dataname refers to either training, test or validation"""
    for dirs,subdr, files in os.walk(dire+category):
        filenames=files
    return filenames

def cleantext(text):
    text=re.sub(r"what's","what is ",text)
    text=re.sub(r"it's","it is ",text)
    text=re.sub(r"\'ve"," have ",text)
    text=re.sub(r"i'm","i am ",text)
    text=re.sub(r"\'re"," are ",text)
    text=re.sub(r"n't"," not ",text)
    text=re.sub(r"\'d"," would ",text)
    text=re.sub(r"\'s","s",text)
    text=re.sub(r"\'ll"," will ",text)
    text=re.sub(r"can't"," cannot ",text)
    text=re.sub(r" e g "," eg ",text)
    text=re.sub(r"e-mail","email",text)
    text=re.sub(r"9\\/11"," 911 ",text)
    text=re.sub(r" u.s"," american ",text)
    text=re.sub(r" u.n"," united nations ",text)
    text=re.sub(r"\n"," ",text)
    text=re.sub(r":"," ",text)
    text=re.sub(r"-"," ",text)
    text=re.sub(r"\_"," ",text)
    text=re.sub(r"\d+"," ",text)
    text=re.sub(r"[$#@%&*!~?%{}()]"," ",text)
    
    return text

def printArticlesum(k):
    print("---------------------original sentence-----------------------")
    print("-------------------------------------------------------------")
    print(data["articles"][k])
    print("----------------------Summary sentence-----------------------")
    print("-------------------------------------------------------------")
    print(data["summaries"][k])
    return 0


def announcedone():
    duration=2000
    freq=440
    ws.Beep(freq,duration)



## Word2vec

In [3]:
def createCorpus(t):
    corpus = []
    all_sent = []
    for k in t:
        for p in t[k]:
            corpus.append(st(p))
    for sent in range(len(corpus)):
        for k in corpus[sent]:
            all_sent.append(k)
    for m in range(len(all_sent)):
        all_sent[m] = wt(all_sent[m])
    
    all_words=[]
    for sent in all_sent:
        hold=[]
        for word in sent:
            hold.append(word.lower())
        all_words.append(hold)
    return all_words

def word2vecmodel(corpus):
    emb_size = emb_size_all
    model_type={"skip_gram":1,"CBOW":0}
    window=10
    workers=4
    min_count=4
    batch_words=20
    epochs=25
    #include bigrams
    #bigramer = gs.models.Phrases(corpus)

    model=gs.models.Word2Vec(corpus,size=emb_size,sg=model_type["skip_gram"],
                             compute_loss=True,window=window,min_count=min_count,workers=workers,
                             batch_words=batch_words)
        
    model.train(corpus,total_examples=len(corpus),epochs=epochs)
#     model.save("%sWord2vec"%modelLocation)
    model.save(ROOT + '/TrainedModels/' + str(epochs) +'_epochs_word2vec_model.h5')

    print('\007')
    return model

def summonehot(corpus):
    allwords=[]
    annotated={}
    for sent in corpus:
        for word in wt(sent):
            allwords.append(word.lower())
    print(len(set(allwords)), "unique characters in corpus")
    #maxcorp=int(input("Enter desired number of vocabulary: "))
    maxcorp=int(len(set(allwords))/1.1)
    wordcount = Counter(allwords).most_common(maxcorp)
    allwords=[]
    
    for p in wordcount:
        allwords.append(p[0])  
        
    allwords=list(set(allwords))
    
    print(len(allwords), "unique characters in corpus after max corpus cut")
    #integer encode
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(allwords)
    #one hot
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    #make look up dict
    for k in range(len(onehot_encoded)): 
        inverted = cleantext(label_encoder.inverse_transform([argmax(onehot_encoded[k, :])])[0]).strip()
        annotated[inverted]=onehot_encoded[k]
    return label_encoder,onehot_encoded,annotated

def wordvecmatrix(model,data):
    IO_data={"article":[],"summaries":[]}
    i=1
    for k in range(len(data["articles"])):
        art=[]
        summ=[]
        for word in wt(data["articles"][k].lower()):
            try:
                art.append(model.wv.word_vec(word))
            except Exception as e:
                e
#                 print(e)

        for word in wt(data["summaries"][k].lower()):
            try:
#                 summ.append(onehot[word])
                summ.append(model.wv.word_vec(word))
            except Exception as e:
                e
#                 print(e)
        
        IO_data["article"].append(art) 
        IO_data["summaries"].append(summ)
        if i%100==0:
            print("progress: " + str(((i*100)/len(data["articles"]))))
        i+=1
    #announcedone()
    print('\007')
    return IO_data

def cutoffSequences(data,artLen,sumlen):
    data2={"article":[],"summaries":[]}
    for k in range(len(data["article"])):
        if len(data["article"][k])<artLen or len(data["summaries"][k])<sumlen:
             #data["article"]=np.delete(data["article"],k,0)
             #data["article"]=np.delete(data["summaries"],k,0)
             pass
        else:
            data2["article"].append(data["article"][k][:artLen])
            data2["summaries"].append(data["summaries"][k][:sumlen])
    return data2


def max_len(data):
    lenk=[]
    for k in data:
        lenk.append(len(k))
    print("The minimum length is: ",min(lenk))
    print("The average length is: ",np.average(lenk))
    print("The max length is: ",max(lenk))
    return min(lenk),max(lenk)

"""reshape vectres for Gensim"""
def reshape(vec):
    return np.reshape(vec,(1,emb_size_all))

def addones(seq):
    return np.insert(seq, [0], [[0],], axis = 0)

def endseq(seq):
    pp=len(seq)
    return np.insert(seq, [pp], [[1],], axis = 0)


## LSTM Model

In [4]:
def encoder_decoder(data):
    print('Encoder_Decoder LSTM...')
   
    """__encoder___"""
    encoder_inputs = Input(shape=en_shape)
    
    encoder_LSTM = LSTM(hidden_units, dropout_U = 0.2, dropout_W = 0.2 ,return_state=True)
    encoder_LSTM_rev=LSTM(hidden_units,return_state=True,go_backwards=True)
    
    #merger=Add()[encoder_LSTM(encoder_inputs), encoder_LSTM_rev(encoder_inputs)]
    encoder_outputsR, state_hR, state_cR = encoder_LSTM_rev(encoder_inputs)
    encoder_outputs, state_h, state_c = encoder_LSTM(encoder_inputs)
    
    state_hfinal=Add()([state_h,state_hR])
    state_cfinal=Add()([state_c,state_cR])
    
    encoder_states = [state_hfinal,state_cfinal]
    
    """____decoder___"""
    decoder_inputs = Input(shape=(None,de_shape[1]))
    decoder_LSTM = LSTM(hidden_units,return_sequences=True,return_state=True)
    decoder_outputs, _, _ = decoder_LSTM(decoder_inputs,initial_state=encoder_states) 
    decoder_dense = Dense(de_shape[1],activation='linear')
    decoder_outputs = decoder_dense(decoder_outputs)
    
    model= Model(inputs=[encoder_inputs,decoder_inputs], outputs=decoder_outputs)
    #plot_model(model, to_file=modelLocation+'model.png', show_shapes=True)
    rmsprop = RMSprop(lr=learning_rate,clipnorm=clip_norm)
    
    model.compile(loss='mse',optimizer=rmsprop,metrics=['accuracy'])

    x_train,x_test,y_train,y_test=tts(data["article"],data["summaries"],test_size=0.20)
    history = model.fit(x=[x_train,y_train],
              y=y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=([x_test,y_test], y_test))
    
    """_________________inference mode__________________"""
    encoder_model_inf = Model(encoder_inputs,encoder_states)
    
    decoder_state_input_H = Input(shape=(hidden_units,))
    decoder_state_input_C = Input(shape=(hidden_units,)) 
    decoder_state_inputs = [decoder_state_input_H, decoder_state_input_C]
    decoder_outputs, decoder_state_h, decoder_state_c = decoder_LSTM(decoder_inputs,
                                                                     initial_state=decoder_state_inputs)
    decoder_states = [decoder_state_h, decoder_state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    
    decoder_model_inf= Model([decoder_inputs]+decoder_state_inputs,
                         [decoder_outputs]+decoder_states)
    
    
    #plot_model(encoder_model_inf, to_file='encoder_model.png', show_shapes=True)
    #plot_model(decoder_model_inf, to_file='decoder_model.png', show_shapes=True)
    scores = model.evaluate([x_test,y_test],y_test, verbose=0)
    
    
    print('LSTM test scores:', scores)
    #announcedone()
    print('\007')
    print(model.summary())
    return model,encoder_model_inf,decoder_model_inf,history

"""___pred____"""
def comparePred(index):
    pred=trained_model.predict([np.reshape(train_data["article"][index],(1,en_shape[0],emb_size_all)),np.reshape(train_data["summaries"][index],(1,de_shape[0],emb_size_all))])
    return pred


"""____generate summary from vectors and remove padding words___"""
def generateText(SentOfVecs):
    SentOfVecs=np.reshape(SentOfVecs,de_shape)
    kk=""
    for k in SentOfVecs:
        kk = kk + label_encoder.inverse_transform([argmax(k)])[0].strip()+" "
#         kk=kk+((getWord(k)[0]+" ") if getWord(k)[1]>0.2 else "")
    return kk

"""___generate summary vectors___"""

def summarize(article):
    stop_pred = False
    article =  np.reshape(article,(1,en_shape[0],en_shape[1]))
    
    #get initial h and c values from encoder
    init_state_val = encoder.predict(article)
    target_seq = np.zeros((1,1,emb_size_all))
    
    generated_summary=[]
    while not stop_pred:
        decoder_out,decoder_h,decoder_c= decoder.predict(x=[target_seq]+init_state_val)
        generated_summary.append(decoder_out)
        init_state_val= [decoder_h,decoder_c]
        #get most similar word and put in line to be input in next timestep
        #target_seq=np.reshape(model.wv[getWord(decoder_out)[0]],(1,1,emb_size_all))
        target_seq=np.reshape(decoder_out,(1,1,emb_size_all))
        if len(generated_summary)== de_shape[0]:
            stop_pred=True
            break
    return generated_summary

"""__________________Plot training curves_______________"""

def plot_training(history):
    print(history.history.keys())
    #  "Accuracy"
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    # "Loss"
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

def evaluate_summ(article):
    ref=''
    for k in wt(data['summaries'][article])[:20]:
        ref=ref+' '+k
    gen_sum = generateText(summarize(new_train_data["article"][article]))
    print("-----------------------------------------------------")
    print("Original summary")
    print(ref)
    print("-----------------------------------------------------")
    print("Generated summary")
    print(gen_sum)
    print("-----------------------------------------------------")

## Sentiment Analysis

In [56]:
from textblob import TextBlob  #Sentiment Analysis
def fill_sentiments(data):
    data['articles'] = 0.0
    data['summaries'] = 0.0
    for arti,summ in data.iterrows():
        cleaned_tweet = row['text']
        s_analysis = TextBlob(cleaned_tweet)
        tweet_df.at[index,'tweet_sentiment_polarity'] = s_analysis.sentiment.polarity
        tweet_df.at[index,'tweet_sentiment_subjectivity'] = s_analysis.sentiment.subjectivity
    return tweet_df

'''Word count of text: To divide the AFINN score with no. of words.'''
def split(delimiters, string, maxsplit=0):
    import re
    regexPattern = '|'.join(map(re.escape, delimiters))
    return re.split(regexPattern, string, maxsplit)

def word_count(string):
    '''Split words in the text delimited by below'''
    delimiters = "."," "
    '''Calculate the number of words in a string'''
    return len(split(delimiters, string, maxsplit=0))



Below is commented out as we are not using this currently

In [8]:
# # load from file into dictionaries
# import pickle
# cnn_stories = pickle.load(open(ROOT +'/cnn_dataset.pkl', 'rb'))
# dm_stories = pickle.load(open(ROOT +'/dm_dataset.pkl', 'rb'))
# cnn_slen = len(cnn_stories)
# dm_slen = len(dm_stories)
# print(cnn_slen,"\n",dm_slen)

# cnn_stories[0]['highlights']

# dm_stories[0]['highlights']

In [41]:
# """
# Using CountVectorizer to extract sentence vectors and unique words/features
# """

# from sklearn.feature_extraction.text import CountVectorizer

# X = [] # X[i].toarray().shape gives (no. of sentences in article/summary i, no. of unique words in all sentences )
# f = [] # f[i] gives a list of all unique words in aticle/summary i (doesn't include stop words)

# for i in range(len(corpus)):
#     if (len(corpus[i])>0):
#         vectorizer = CountVectorizer()
#         X.append(vectorizer.fit_transform(corpus[i]))
#         f.append(vectorizer.get_feature_names())
#     else:
#         print("Encountered empty line - Skipped at", i)

# # # To obtain the no. of unique words
# # label_encoder,onehot_encoded,onehot=summonehot(data["summaries"])

# Functions for republic.ipynb summarization

In [1]:
import string
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences


# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

# turn a doc into clean tokens
def clean_doc(doc):
    # replace '--' with a space ' '
    doc = doc.replace('--', ' ')
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # make lower case
    tokens = [word.lower() for word in tokens]
    return tokens

# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()
    
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    # select a seed text
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # the seed text must be encoded to integers using the same tokenizer that we used when training the model.
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)

Using TensorFlow backend.
